In [1]:
import MeCab as mc
import re
import urllib.request
import unicodedata
import pandas as pd
import numpy as np
import math

In [2]:
#MeCabによる形態素解析
def extractter(text):
    text = unicodedata.normalize("NFKC", text)
    tagger = mc.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    key= []
    word = []
    keyword =[]
    while node:
        nes = node.feature.split(",")
        wor = node.surface
        if wor != "":
            nes = node.feature.split(",")
            if nes[0] == u"名詞":
                if nes[6] == u"*":
                     keyword.append(wor)
                else:
                    keyword.append(nes[6])
        node = node.next
        if node is None:
            break
    return keyword

#slothlibのストップワードの取得
def get_stopword():
    slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    slothlib_file = urllib.request.urlopen(slothlib_path)
    slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
    slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']
    return slothlib_stopwords

#ストップワードの除去
def except_stopwords(text):
    stopwords = get_stopword()
    for i in text:
        for j in stopwords:
            if i == j:
                text.remove(j)
    return text

In [3]:
#データの確認
base_data = pd.read_csv(filepath_or_buffer="Datas/pickup_data.csv", encoding="utf_8", sep=",")
base_data.info()
base_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
screen_name    20000 non-null object
user_id        20000 non-null float64
tweet_id       20000 non-null float64
tweet          20000 non-null object
tweet2         20000 non-null object
postdate       20000 non-null object
cos_day        20000 non-null float64
sin_day        20000 non-null float64
tag            20000 non-null object
image_url      20000 non-null object
image          20000 non-null int64
retweet        20000 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 1.8+ MB


,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet
0,遠野物語を紐解く,7.400501e+17,8.254434e+17,lt 姥子淵の河童 gt 姥子淵の近くの家が子供に頼んで淵へ馬の足を冷やしに行かせたが 子...,lt 姥 淵 河童 gt 姥 淵 家 子供 淵 馬の足 子供 途中 遊び その間 河童 馬 ...,2017-01-29,0.478734,0.877960,0,http://pbs.twimg.com/media/Cj3NdqZUgAAdZR5.jpg...,1,1.0
1,ココよちゃん＠いわて【岩手の宿】,3.010894e+09,1.034326e+18,いわて八幡平 旬菜フレンチと貸切温泉の小さな宿 Beaver ビーバー 旬菜フレンチ 貸切露...,八幡平 旬 菜 フレンチ 貸切 温泉 宿 Beaver ビーバー 旬 菜 フレンチ 貸切 露...,2018-08-28,-0.835925,-0.548843,0,0,0,0.0
2,☆たかたのゆめちゃん☆（公式）,1.029720e+09,7.158523e+17,だまされた 嘘がかわいい きょうだけは ゆめ あのカレ...,嘘 きょう カレー の おわび 市役所 玄関 花 あした さし みんな キラ ん,2016-04-01,0.999917,-0.012910,0,http://pbs.twimg.com/media/Ce834G-WEAAQEty.jpg...,1,0.0
3,花巻市,2.843129e+09,8.414505e+17,賢治の短歌百首カルタ をご利用ください 宮沢賢治生誕 年の平成 年に 記念事業の一...,賢治 短歌 首 カルタ 利用 宮沢 賢治 生誕 平成 記念 事業 一環 作成 市内 図書館 ...,2017-03-14,0.951057,0.309017,0,0,0,1.0
4,花巻市,2.843129e+09,1.041864e+18,月 日から 日まで秋の全国交通安全運動が実施されます 期間中は交通安全を呼びかける...,秋 全国 交通 安全 運動 実施 期間 交通 安全 街頭 活動 実施 運動 機会 交通 安全...,2018-09-18,-0.976011,-0.217723,0,http://pbs.twimg.com/media/DnVxh1RUwAAyU3I.jpg...,1,1.0


In [4]:
#user_idの固有データの取り出し
nu = base_data['user_id'].unique()
print(nu)
#データフレームからuser_idを抽出
ids = base_data['user_id']
#0~31の数値へ変換
post_user = []
for i in ids:
    for j in range(len(nu)):
        if i == nu[j]:
            post_user.append(j)

#データフレームへ変換
s = pd.DataFrame(post_user, columns=['user_id2'])
#元のデータと結合
#n_base_data = base_data.join(s)

[7.40050123e+17 3.01089352e+09 1.02971993e+09 2.84312922e+09
 5.43138610e+08 7.48485695e+17 1.02589448e+08 8.93037779e+17
 3.13971513e+09 2.22008119e+08 1.29824840e+08 9.56572308e+08
 7.58099901e+17 1.96899097e+08 1.73378242e+09 5.82425732e+08
 1.33582577e+08 2.70280216e+08 2.45955083e+09 1.37168394e+09
 9.45626465e+17 9.95901925e+17 7.60852304e+08 8.52638190e+07
 2.32480106e+09 9.73451829e+17 9.71187020e+07 2.75767513e+09
 2.64482835e+08 4.56310285e+09 1.61550679e+09 1.66203998e+09]


In [5]:
#不要な文字の削除
tag_list = base_data["tag"].str.replace(r'[0-9]', ' ').str.replace(r'[０-９]', ' ').str.replace(r'([^\s\w])+', ' ')
#形態素解析
for i in range(len(tag_list)):
    tag_list[i] = extractter(tag_list[i])
    tag_list[i] = except_stopwords(tag_list[i])

In [6]:
tag_list[2]

[]

In [7]:
#元のデータと結合
n_base_data = base_data.join(s)
n_base_data['tag2'] = tag_list

In [10]:
n_base_data.head()

,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet,user_id2,tag2
0,遠野物語を紐解く,7.400501e+17,8.254434e+17,lt 姥子淵の河童 gt 姥子淵の近くの家が子供に頼んで淵へ馬の足を冷やしに行かせたが 子...,lt 姥 淵 河童 gt 姥 淵 家 子供 淵 馬の足 子供 途中 遊び その間 河童 馬 ...,2017-01-29,0.478734,0.877960,0,http://pbs.twimg.com/media/Cj3NdqZUgAAdZR5.jpg...,1,1.0,0,[]
1,ココよちゃん＠いわて【岩手の宿】,3.010894e+09,1.034326e+18,いわて八幡平 旬菜フレンチと貸切温泉の小さな宿 Beaver ビーバー 旬菜フレンチ 貸切露...,八幡平 旬 菜 フレンチ 貸切 温泉 宿 Beaver ビーバー 旬 菜 フレンチ 貸切 露...,2018-08-28,-0.835925,-0.548843,0,0,0,0.0,1,[]
2,☆たかたのゆめちゃん☆（公式）,1.029720e+09,7.158523e+17,だまされた 嘘がかわいい きょうだけは ゆめ あのカレ...,嘘 きょう カレー の おわび 市役所 玄関 花 あした さし みんな キラ ん,2016-04-01,0.999917,-0.012910,0,http://pbs.twimg.com/media/Ce834G-WEAAQEty.jpg...,1,0.0,2,[]
3,花巻市,2.843129e+09,8.414505e+17,賢治の短歌百首カルタ をご利用ください 宮沢賢治生誕 年の平成 年に 記念事業の一...,賢治 短歌 首 カルタ 利用 宮沢 賢治 生誕 平成 記念 事業 一環 作成 市内 図書館 ...,2017-03-14,0.951057,0.309017,0,0,0,1.0,3,[]
4,花巻市,2.843129e+09,1.041864e+18,月 日から 日まで秋の全国交通安全運動が実施されます 期間中は交通安全を呼びかける...,秋 全国 交通 安全 運動 実施 期間 交通 安全 街頭 活動 実施 運動 機会 交通 安全...,2018-09-18,-0.976011,-0.217723,0,http://pbs.twimg.com/media/DnVxh1RUwAAyU3I.jpg...,1,1.0,3,[]


In [11]:
#データ一時保存
n_base_data.to_csv("Datas/multi_data.csv",index=False, sep=",")
#データの確認
b_data = pd.read_csv(filepath_or_buffer="Datas/multi_data.csv", encoding="utf_8", sep=",")
b_data.info()
b_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
screen_name    20000 non-null object
user_id        20000 non-null float64
tweet_id       20000 non-null float64
tweet          20000 non-null object
tweet2         20000 non-null object
postdate       20000 non-null object
cos_day        20000 non-null float64
sin_day        20000 non-null float64
tag            20000 non-null object
image_url      20000 non-null object
image          20000 non-null int64
retweet        20000 non-null float64
user_id2       20000 non-null int64
tag2           20000 non-null object
dtypes: float64(5), int64(2), object(7)
memory usage: 2.1+ MB


,screen_name,user_id,tweet_id,tweet,tweet2,postdate,cos_day,sin_day,tag,image_url,image,retweet,user_id2,tag2
0,遠野物語を紐解く,7.400501e+17,8.254434e+17,lt 姥子淵の河童 gt 姥子淵の近くの家が子供に頼んで淵へ馬の足を冷やしに行かせたが 子...,lt 姥 淵 河童 gt 姥 淵 家 子供 淵 馬の足 子供 途中 遊び その間 河童 馬 ...,2017-01-29,0.478734,0.877960,0,http://pbs.twimg.com/media/Cj3NdqZUgAAdZR5.jpg...,1,1.0,0,[]
1,ココよちゃん＠いわて【岩手の宿】,3.010894e+09,1.034326e+18,いわて八幡平 旬菜フレンチと貸切温泉の小さな宿 Beaver ビーバー 旬菜フレンチ 貸切露...,八幡平 旬 菜 フレンチ 貸切 温泉 宿 Beaver ビーバー 旬 菜 フレンチ 貸切 露...,2018-08-28,-0.835925,-0.548843,0,0,0,0.0,1,[]
2,☆たかたのゆめちゃん☆（公式）,1.029720e+09,7.158523e+17,だまされた 嘘がかわいい きょうだけは ゆめ あのカレ...,嘘 きょう カレー の おわび 市役所 玄関 花 あした さし みんな キラ ん,2016-04-01,0.999917,-0.012910,0,http://pbs.twimg.com/media/Ce834G-WEAAQEty.jpg...,1,0.0,2,[]
3,花巻市,2.843129e+09,8.414505e+17,賢治の短歌百首カルタ をご利用ください 宮沢賢治生誕 年の平成 年に 記念事業の一...,賢治 短歌 首 カルタ 利用 宮沢 賢治 生誕 平成 記念 事業 一環 作成 市内 図書館 ...,2017-03-14,0.951057,0.309017,0,0,0,1.0,3,[]
4,花巻市,2.843129e+09,1.041864e+18,月 日から 日まで秋の全国交通安全運動が実施されます 期間中は交通安全を呼びかける...,秋 全国 交通 安全 運動 実施 期間 交通 安全 街頭 活動 実施 運動 機会 交通 安全...,2018-09-18,-0.976011,-0.217723,0,http://pbs.twimg.com/media/DnVxh1RUwAAyU3I.jpg...,1,1.0,3,[]


In [12]:
#不要文字の置き換え
b_data['tag2'] = b_data['tag2'].str.replace(r'([^\s\w])+', '')

In [13]:
#データ再保存
b_data.to_csv("Datas/multi_data.csv",index=False, sep=",")